In [4]:
import cv2
import numpy as np
import time

import mediapipe as mp
mp_objectron = mp.solutions.objectron
mp_drawing = mp.solutions.drawing_utils


#shoe_images = {'image': cv2.imread('shoe.png')}
#cv2.imshow('image', shoe_images['image'])
vid = cv2.VideoCapture(0)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)



while True:

  _, img = vid.read()
  shoe_images = {'image': img}
  with mp_objectron.Objectron(
      static_image_mode=True,
      max_num_objects=6,
      min_detection_confidence=0.5,
      model_name='Shoe') as objectron:
    # Run inference on shoe images.
    for name, image in shoe_images.items():
      # Convert the BGR image to RGB and process it with MediaPipe Objectron.
      results = objectron.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

      # Draw box landmarks.
      if not results.detected_objects:
        print(f'No box landmarks detected on {name}')
        continue
      print(f'Box landmarks of {name}:')
      annotated_image = image.copy()
      for detected_object in results.detected_objects:
        mp_drawing.draw_landmarks( annotated_image, detected_object.landmarks_2d, mp_objectron.BOX_CONNECTIONS)
        mp_drawing.draw_axis(annotated_image, detected_object.rotation, detected_object.translation)
      cv2.imshow('img', annotated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
cv2.waitKey(0)
cv2.destroyAllWindows()

[ WARN:0@30.015] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@30.015] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [1]:
import cv2
import time

vid = cv2.VideoCapture(0)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)


while True:
    _, img = vid.read()
    cv2.imshow('image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
import cv2
import numpy as np
import time

import mediapipe as mp

img = cv2.imread('shoe.png')
img = cv2.imread('/home/hameed/Downloads/shoes.jpeg')
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
h = np.load('h.npy')
def midpoint(x1, y1, x2, y2):
    return ((x1 + x2)/2, (y1 + y2)/2)

mp_objectron = mp.solutions.objectron

fx, fy = (1.0, 1.0)
px, py = (0.0, 0.0)
iterations = [1, 2, 5, 6]
objectron = mp_objectron.Objectron(mp_objectron.Objectron(static_image_mode=True, max_num_objects=5, min_detection_confidence=0.5, model_name='Shoe'))
results = objectron.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ann_img = img.copy()
num_obj = len(results.detected_objects)
x = []
y = []
z_axis = []
axis_world = np.float32([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
for obj in results.detected_objects:

    x.append(int(obj.landmarks_2d.landmark[0].x * img.shape[1]))
    y.append(int(obj.landmarks_2d.landmark[0].y * img.shape[0]))

    axis_cam = np.matmul(obj.rotation, 0.1 * axis_world.T).T + obj.translation
    x_ori = axis_cam[..., 0]
    y_ori = axis_cam[..., 1]
    z_ori = axis_cam[..., 2]
# Project 3D points to NDC space.
    x_ndc = np.clip(-fx * x_ori / (z_ori + 1e-5) + px, -1., 1.)
    y_ndc = np.clip(-fy * y_ori / (z_ori + 1e-5) + py, -1., 1.)
# Convert from NDC space to image space.
    x_im = np.int32((1 + x_ndc) * 0.5 * img.shape[1])
    y_im = np.int32((1 - y_ndc) * 0.5 * img.shape[0])
    z_axis.append((x_im[3], y_im[3]))

try:
    x, y = midpoint(x[0], y[0], x[1], y[1])
    z0, z1 = midpoint(z_axis[0][0], z_axis[0][1], z_axis[1][0], z_axis[1][1])
    ann_img = cv2.circle(ann_img, (int(x), int(y)), radius=100, color=(0, 0, 255), thickness=5)
    ann_img = cv2.arrowedLine(ann_img, (int(x), int(y)), (int(z0), int(z1)), color=(0, 128, 0), thickness=3, tipLength=0.5)
    warp_img = cv2.warpPerspective(ann_img, h, (img.shape[1],img.shape[0]))
except:
    print('not enough points')

cv2.imshow( 'ann_img', warp_img)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [13]:
(int(z0), int(z1))

(147, 117)

In [40]:
import cv2
import numpy as np
import time

import mediapipe as mp

img = cv2.imread('shoe.png')
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

mp_objectron = mp.solutions.objectron
mp_drawing = mp.solutions.drawing_utils

objectron = mp_objectron.Objectron(mp_objectron.Objectron(static_image_mode=True, max_num_objects=5, min_detection_confidence=0.5, model_name='Shoe'))
results = objectron.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ann_img = img.copy()
detected_object = results.detected_objects[0]
mp_drawing.draw_landmarks( ann_img, detected_object.landmarks_2d, mp_objectron.BOX_CONNECTIONS)
mp_drawing.draw_axis( ann_img, detected_object.rotation, detected_object.translation)
cv2.imshow( 'ann_img', ann_img)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [47]:

axis_world = np.float32([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
x = axis_world[..., 3]
x

IndexError: index 3 is out of bounds for axis 1 with size 3

In [45]:
axis_world

array([[0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [80]:
iterations = [1, 2, 5, 6]
ann_img = img.copy()
for i in iterations:
    x = int(detected_object.landmarks_2d.landmark[i].x * img.shape[1])
    y = int(detected_object.landmarks_2d.landmark[i].y * img.shape[0])
    ann_img = cv2.circle(ann_img, (x,y), 1, color=(0, 0, 255), thickness=5)
cv2.imshow(f'img{i}', ann_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [73]:
range(9)

range(0, 9)

In [5]:
import cv2
import numpy as np
import time

import mediapipe as mp
mp_objectron = mp.solutions.objectron
mp_drawing = mp.solutions.drawing_utils


#shoe_images = {'image': cv2.imread('shoe.png')}
#cv2.imshow('image', shoe_images['image'])
vid = cv2.VideoCapture(0)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)



while True:

  _, img = vid.read()
  with mp_objectron.Objectron(static_image_mode=True, max_num_objects=6, min_detection_confidence=0.5, model_name='Shoe') as objectron:
   
    # Convert the BGR image to RGB and process it with MediaPipe Objectron.
    results = objectron.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # Draw box landmarks.
    if not results.detected_objects:
      print('No box landmarks detected on shoe')
      continue
    print('Box landmarks of shoe:')
    annotated_image = img.copy()
    for detected_object in results.detected_objects:
      mp_drawing.draw_landmarks( annotated_image, detected_object.landmarks_2d, mp_objectron.BOX_CONNECTIONS)
      #mp_drawing.draw_axis(annotated_image, detected_object.rotation, detected_object.translation)
    cv2.imshow('img', annotated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
cv2.waitKey(0)
cv2.destroyAllWindows()

No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmar

: 

: 

In [19]:
try:
  print(pppp)
except:
  print("An exception occurred")

An exception occurred


In [20]:
try:
  print("Hello")
except:
  print("Something went wrong")
else:
  print("Nothing went wrong")

Hello
Nothing went wrong
